In [1]:
from Recommenders.Ensambles import LinearCombination
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [2]:
import pandas as pd
import matplotlib.pyplot as pyplot
import numpy as np

In [3]:
np.int = int
np.float = float 
np.bool = bool

In [15]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: 'c:\Users\melan\AppData\Local\Programs\Python\Python311\python.exe'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... PASS

Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... 
Compiling [2/10]: MatrixFactorization_Cython_Epoch.pyx... PASS

Compiling [3/10]: Compute_Similarity_Cython.pyx... 
Compiling [3/10]: Compute_Similarity_Cython.pyx... PASS

Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... 
Compiling [4/10]: SLIM_BPR_Cython_Epoch.pyx... FAIL: Command 'c:\Users\melan\AppData\Local\Programs\Python\Python311\python.exe c:\Users\melan\shared-folder\RecSys\RecSys-Competition/CythonCompiler/compile_script.py SLIM_BPR_Cython_Epoch.pyx build_ext --inplace' returned non-zero exit status 1.

Compiling [5/10]: Sparse_Matrix_Tree_CSR.pyx... 
Compiling

C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\Users\melan\shared-folder\RecSys\RecSys-Competition\Recommenders\MatrixFactorization\Cython\MatrixFactorizationImpressions_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\Users\melan\shared-folder\RecSys\RecSys-Competition\Recommenders\MatrixFactorization\Cython\MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\melan\AppData\Local\Programs\Python\Python311\Lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' no

## Import Dataset

In [4]:
URM_all_dataframe = pd.read_csv('data_train.csv', 
                                sep=",", 
                                header= 0, 
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [5]:
# Move to sparse format
import scipy.sparse as sps

URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values))) ## .values --> numpy array, df[..] --> pd series

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 3076 (23.6%) Users that have less than 1 test interactions


## kNN-Ensamble

In [7]:
# Initialize the two recommenders for the ensamble
kNN_user = UserKNNCFRecommender(URM_train)
kNN_item = ItemKNNCFRecommender(URM_train)

kNN_list = [kNN_user, kNN_item]

UserKNNCFRecommender: URM Detected 551 ( 4.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 199 ( 0.9%) items with no interactions.
ItemKNNCFRecommender: URM Detected 551 ( 4.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 199 ( 0.9%) items with no interactions.


In [8]:
# Create hyperparameters list
hyperp_user = {
    "topK": 50, 
    "shrink": 100, 
    "similarity": 'cosine', 
    "normalize": True, 
    "feature_weighting": "none", 
    "URM_bias": False,
}

hyperp_item = {
    "topK": 50, 
    "shrink": 100, 
    "similarity": 'cosine', 
    "normalize": True, 
    "feature_weighting": "none", 
    "URM_bias": False,
}

hyperp_list = [hyperp_user, hyperp_item]

In [9]:
# Instantiate the Ensamble
# If you don't specify the weights, the contribute of each Recommender will be the same.
kNNHybrid = LinearCombination(URM_train, recommenders_list= kNN_list, hyperparameters_dicts_list= hyperp_list)

Linear_Combination_Ensamble_Recommender_Class: URM Detected 551 ( 4.2%) users with no interactions.
Linear_Combination_Ensamble_Recommender_Class: URM Detected 199 ( 0.9%) items with no interactions.


In [10]:
# Fit the models in the ensamble
kNNHybrid.fit()

Unable to load Cython Compute_Similarity, reverting to Python


Similarity column 13025 (100.0%), 864.23 column/sec. Elapsed time 15.07 sec
Successfully fitted Recommender 1 :  UserKNNCFRecommender
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 819.67 column/sec. Elapsed time 27.26 sec
Successfully fitted Recommender 2 :  ItemKNNCFRecommender


In [14]:
# Make recommendations
kNNHybrid.recommend([2,3,4,5])

[0.5, 0.5]
(2, 4, 22348)
(4, 22348)


[[7,
  19,
  42,
  2,
  5,
  4,
  17,
  3,
  15,
  12,
  9,
  6,
  16,
  81,
  78,
  51,
  50,
  359,
  44,
  14,
  131,
  8,
  26,
  31,
  18,
  229,
  22,
  108,
  90,
  52,
  29,
  69,
  47,
  155,
  949,
  114,
  227,
  11,
  23,
  222,
  13,
  72,
  136,
  276,
  88,
  85,
  290,
  28,
  137,
  62,
  54,
  75,
  21,
  105,
  125,
  130,
  68,
  345,
  32,
  33,
  157,
  158,
  67,
  365,
  80,
  154,
  271,
  221,
  101,
  264,
  121,
  159,
  46,
  30,
  118,
  416,
  422,
  435,
  196,
  300,
  39,
  38,
  338,
  263,
  100,
  199,
  104,
  84,
  140,
  192,
  58,
  174,
  656,
  402,
  20,
  180,
  324,
  102,
  203,
  380,
  289,
  132,
  117,
  237,
  134,
  191,
  167,
  57,
  629,
  96,
  283,
  152,
  475,
  178,
  61,
  223,
  65,
  250,
  206,
  177,
  173,
  92,
  737,
  37,
  48,
  128,
  272,
  317,
  150,
  1095,
  93,
  525,
  277,
  238,
  498,
  1227,
  284,
  79,
  260,
  45,
  322,
  286,
  41,
  202,
  215,
  209,
  311,
  188,
  3069,
  549,
  145,
  241,
  31

## Linear combination hyperparameter tuning with Optuna


In [19]:
import optuna

class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result", "train_time (min)"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        if not isinstance(self.results_df, pd.DataFrame):
            self.results_df = pd.DataFrame()  # Reinitialize as DataFrame if not already
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)

In [20]:
def objective_function_KNN_ensamble(optuna_trial):
    # Initialize the two recommenders for the ensamble
    kNN_user = UserKNNCFRecommender(URM_train)
    kNN_item = ItemKNNCFRecommender(URM_train)

    kNN_list = [kNN_user, kNN_item]
    
    # Set hyperparameters for each of the methods in the Ensamble
    # Maybe tune the fact of using merge_topPop before or after the ensamble
    similarity_user = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    similarity_item = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])

    hyperp_user = {
    "topK": optuna_trial.suggest_int("topK_user", 5, 1000),
    "shrink": optuna_trial.suggest_int("shrink_user", 0, 1000),
    "similarity": similarity_user, 
    "normalize": optuna_trial.suggest_categorical("normalize_user", [True, False]),
    "feature_weighting": optuna_trial.suggest_categorical("feature_weighting_user"["BM25", "TF-IDF", "none"]),
    "URM_bias": optuna_trial.suggest_categorical("URM_bias_user", [True, False]),
    "merge_topPop": optuna_trial.suggest_categorical("merge_topPop_user", [True, False]), 
    }
    if hyperp_item["merge_topPop"]:
        hyperp_item["topPop_factor"] = optuna_trial.suggest_float("topPop_factor_user", 1e-8, 1e-1, log=True)
    if similarity_user == "asymmetric":
        hyperp_user["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha_user", 0, 2, log=False)
        hyperp_user["normalize"] = True     
    elif similarity_user == "tversky":
        hyperp_user["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha_user", 0, 2, log=False)
        hyperp_user["tversky_beta"] = optuna_trial.suggest_float("tversky_beta_user", 0, 2, log=False)
        hyperp_user["normalize"] = True 
    elif similarity_user == "euclidean":
        hyperp_user["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row_user", [True, False])
        hyperp_user["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode_user", ["lin", "log", "exp"])

    hyperp_item = {
        "topK": optuna_trial.suggest_int("topK_item", 5, 1000),
        "shrink": optuna_trial.suggest_int("shrink_item", 0, 1000),
        "similarity": similarity_item, 
        "normalize": optuna_trial.suggest_categorical("normalize_item", [True, False]),
        "feature_weighting": optuna_trial.suggest_categorical("feature_weighting_item"["BM25", "TF-IDF", "none"]),
        "URM_bias": optuna_trial.suggest_categorical("URM_bias_item", [True, False]),
        "merge_topPop": optuna_trial.suggest_categorical("merge_topPop_item", [True, False]), 
    }
    if hyperp_item["merge_topPop"]:
        hyperp_item["topPop_factor"] = optuna_trial.suggest_float("topPop_factor_item", 1e-8, 1e-1, log=True)
    if similarity_item == "asymmetric":
        hyperp_item["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha_item", 0, 2, log=False)
        hyperp_item["normalize"] = True     
    elif similarity_item == "tversky":
        hyperp_item["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha_item", 0, 2, log=False)
        hyperp_item["tversky_beta"] = optuna_trial.suggest_float("tversky_beta_item", 0, 2, log=False)
        hyperp_item["normalize"] = True 
    elif similarity_item == "euclidean":
        hyperp_item["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row_item", [True, False])
        hyperp_item["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode_item", ["lin", "log", "exp"])

    hyperp_list = [hyperp_user, hyperp_item]

    # Set the weights of the Ensamble. Optuna will take care of tuning them
    userkNN_contribute = optuna_trial.suggest_float("userkNN_contribute", 0, 1)
    itemkNN_contribute = optuna_trial.suggest_float("itemkNN_contribute", 0, 1)
    weights_list = [userkNN_contribute, itemkNN_contribute]

    # Instantiate the Ensamble
    recommender_instance = LinearCombination(URM_train, 
                                             recommenders_list= kNN_list, 
                                             hyperparameters_dicts_list= hyperp_list,
                                             weights_list= weights_list)    
    
    merge_topPop_ensamble = optuna_trial.suggest_categorical("merge_topPop_ensamble", [True, False])
    if merge_topPop_ensamble:
        topPop_factor_ensamble = optuna_trial.suggest_float("topPop_factor_ensamble", 1e-8, 1e-1, log=True)
    else: 
        topPop_factor_ensamble = 0.0
    recommender_instance.fit(merge_topPop= merge_topPop_ensamble, topPop_factor= topPop_factor_ensamble)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP_MIN_DEN"]

In [ ]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_KNN_ensamble,
                      callbacks=[save_results],
                      n_trials = 1000)

In [ ]:
# recommender_instance = LinearCombination(URM_train + URM_validation, ...)
# recommender_instance.fit(**optuna_study.best_trial.params)

#result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
#result_df